<a href="https://colab.research.google.com/github/olusolaag/AI-ML-Projects-2021/blob/main/sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Movie review sentiment analysis using IMDB dataset and keras (deep learning)

In [2]:
!pip install kaggle

In [3]:
!ls

sample_data


In [4]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"olusolaag","key":"9a421f26f1a18c84f0bb87afe5493d83"}'}

In [5]:
!pip install -q kaggle

In [6]:
!mkdir -p ~/.kaggle

In [7]:
!cp kaggle.json ~/.kaggle

In [8]:
!ls ~/.kaggle

kaggle.json


In [9]:
!chmod 600 /root/.kaggle/kaggle.json

In [10]:
!mkdir sent-analysis

In [11]:
!kaggle datasets download -d lakshmi25npathi/imdb-dataset-of-50k-movie-reviews -p sent-analysis

 86% 22.0M/25.7M [00:00<00:00, 14.8MB/s]
100% 25.7M/25.7M [00:00<00:00, 40.6MB/s]


In [12]:
!unzip /content/sent-analysis/imdb-dataset-of-50k-movie-reviews.zip

Archive:  /content/sent-analysis/imdb-dataset-of-50k-movie-reviews.zip
  inflating: IMDB Dataset.csv        


In [13]:
! pip install contractions
! pip install inflect


     |████████████████████████████████| 283 kB 4.1 MB/s 
     |████████████████████████████████| 321 kB 41.5 MB/s 
  Created wheel for pyahocorasick: filename=pyahocorasick-1.4.2-cp37-cp37m-linux_x86_64.whl size=85459 sha256=03285e76d278c8d0ed07ec7f0584f1f03b9a7a577fd56f3874040d09dc6782d8
  Stored in directory: /root/.cache/pip/wheels/25/19/a6/8f363d9939162782bb8439d886469756271abc01f76fbd790f
Successfully built pyahocorasick


In [14]:
import pandas as pd
import numpy as np
import re
import string
from numpy import array
import contractions # to remove contractions like don't
import inflect # to convert digits to text
from numpy import array
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import GlobalMaxPooling1D
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer

In [15]:
# load dataset and check for null values
movie_reviews = pd.read_csv("/content/IMDB Dataset.csv")

movie_reviews.isnull().values.any()

False

In [16]:
print(movie_reviews.shape)
movie_reviews.head()

(50000, 2)


,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [17]:
# Let us view one of the reviews
movie_reviews["review"][3]

"Basically there's a family where a little boy (Jake) thinks there's a zombie in his closet & his parents are fighting all the time.<br /><br />This movie is slower than a soap opera... and suddenly, Jake decides to become Rambo and kill the zombie.<br /><br />OK, first of all when you're going to make a film you must Decide if its a thriller or a drama! As a drama the movie is watchable. Parents are divorcing & arguing like in real life. And then we have Jake with his closet which totally ruins all the film! I expected to see a BOOGEYMAN similar movie, and instead i watched a drama with some meaningless thriller spots.<br /><br />3 out of 10 just for the well playing parents & descent dialogs. As for the shots with Jake: just ignore them."

In [18]:
# create a function to remove html tags, punctuations, special characters etc. from the review text

# def remove_round_brackets(data): # function remove backets and special characters
#   return re.sub('\(.*?#@%&!\)','',data)



def number_to_text(data): #function converts digits to word
  temp_str = data.split()
  string = []
  for i in temp_str:
    # if the word is digit, converted to 
    # word else the sequence continues
    if i.isdigit():
      temp = inflect.engine().number_to_words(i)
      string.append(temp)
    else:
      string.append(i)
  return temp_str


def preprocess_text(sen):
    # PRE-PROCESS A GIVEN REVIEW
    # sen = ' '.join(map(str, sen))
    sen = re.sub('<.*?>', ' ', sen) # remove html tag
    sen = re.sub(r'https\S', ' ', sen) # remove url
    sen = re.sub(r'[^\w\s]', '', sen) #
    # sen = re.sub('[^a-zA-Z]', ' ', sen)    # remove non alphabet. This removes digits which are also essential
    sen = contractions.fix(sen)
    sen = re.sub(r"\s+[a-zA-Z]\s+", ' ', sen)    # remove Single characters
    sen = re.sub(r'\s+', ' ', sen)     # remove multiple spaces
    sen = sen.lower() # lower case
    return sen

In [19]:
# Store the preprocessed reviews in a new list
X = []
sentences = list(movie_reviews['review'])

for sen in sentences:
  
  X.append(preprocess_text(sen))

In [20]:
print(X[3])

basically there is family where little boy jake thinks there is zombie in his closet his parents are fighting all the time this movie is slower than soap opera and suddenly jake decides to become rambo and kill the zombie ok first of all when you are going to make film you must decide if its thriller or drama as drama the movie is watchable parents are divorcing arguing like in real life and then we have jake with his closet which totally ruins all the film expected to see boogeyman similar movie and instead watched drama with some meaningless thriller spots 3 out of 10 just for the well playing parents descent dialogs as for the shots with jake just ignore them


In [21]:
# Convert labels to integers
y = movie_reviews['sentiment']
y = np.array(list(map(lambda x: 1 if x=="positive" else 0, y)))

In [22]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

print('Train Set',len(X_train))
print('Test Set',len(X_test))

Train Set 40000
Test Set 10000


In [23]:
print(X_train[3])

this latterday fulci schlocker is totally abysmal concoction dealing with an incurable gambler brett halsey who decides bluebeardstyle to pay off his everrising debts by seducing some of the ugliest bitches you will ever lay your eyes on and who just happen to be wealthy widows the fulcipenned script also contrives to incorporate few blackly comedic elements which only result in some unfunny business involving corpse which will not stay put an opera singer victim who will not stop singing etc not to mention doppelganger theme straight out of the student of prague although in this case the two personas communicate via prerecorded radio messages in the end cannot say am surprised that this film shows no sign of the sophistication of mario bavas hatchet for the honeymoon 1970 which it resembles in several ways and that it is content to merely pile up the disgustingly gory but nonetooconvincing effects of dismembered limbs and squashed or melting faces with which alas fulci had by then bec

In [24]:
# Tokenizer class from the keras.preprocessing.text module creates a word-to-index integer dictionary

tokenizer = Tokenizer(num_words=5000) #nb_words: None or int. Maximum number of words to work with (if set, tokenization will be restricted to the top nb_words most common words in the dataset)

# https://stackoverflow.com/questions/51956000/what-does-keras-tokenizer-method-exactly-do
"""fit_on_texts Updates internal vocabulary based on a list of texts. 
This method creates the vocabulary index based on word frequency. 
So if you give it something like, "The cat sat on the mat." 
It will create a dictionary s.t. word_index["the"] = 1; 
word_index["cat"] = 2 it is word -> index dictionary so every word gets a unique integer value. 
0 is reserved for padding. So lower integer means more 
frequent word (often the first few are stop words because they appear a lot)."""
tokenizer.fit_on_texts(X_train) 
"""texts_to_sequences Transforms each text in texts to a sequence of integers. 
So it basically takes each word in the text and replaces it with its corresponding 
integer value from the word_index dictionary."""

X_train = tokenizer.texts_to_sequences(X_train)
X_test = tokenizer.texts_to_sequences(X_test)

In [25]:
X_train[3]

[8,
 3514,
 5,
 427,
 4368,
 1876,
 14,
 31,
 32,
 1029,
 4,
 978,
 121,
 22,
 30,
 46,
 3,
 1,
 18,
 57,
 118,
 4114,
 119,
 488,
 19,
 2,
 32,
 38,
 534,
 4,
 23,
 2790,
 1,
 217,
 81,
 4,
 163,
 1629,
 746,
 61,
 60,
 897,
 6,
 46,
 1872,
 946,
 1132,
 3368,
 61,
 57,
 10,
 734,
 256,
 31,
 1361,
 1820,
 1379,
 32,
 57,
 10,
 564,
 1112,
 505,
 10,
 4,
 707,
 729,
 767,
 44,
 3,
 1,
 1372,
 3,
 244,
 6,
 8,
 393,
 1,
 107,
 2591,
 1416,
 3404,
 6,
 1,
 122,
 140,
 127,
 93,
 716,
 9,
 8,
 17,
 246,
 55,
 1933,
 3,
 1,
 3,
 3669,
 13,
 1,
 61,
 7,
 3619,
 6,
 412,
 731,
 2,
 9,
 7,
 5,
 1400,
 4,
 1508,
 2548,
 56,
 1,
 2038,
 16,
 273,
 3,
 2,
 39,
 1381,
 14,
 61,
 2971,
 3514,
 66,
 30,
 91,
 407,
 313,
 3333]

In [26]:
print(np.asarray(X_train).shape) #np.asarray - Input data, in any form that can be converted to an array. This includes lists, lists of tuples, tuples, tuples of tuples, tuples of lists and ndarrays.
print(np.asarray(X_test).shape)

(40000,)
(10000,)


/usr/local/lib/python3.7/dist-packages/numpy/core/_asarray.py:83: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  return array(a, dtype, copy=False, order=order)


In [27]:
# Perform padding on both train and test set, Set the maximum length of each list to 100 and add 0 padding to those lists that have a length < 100, until they reach a length of 100
max_length = 100

X_train = pad_sequences(X_train, maxlen=max_length, padding='post', truncating='post')
X_test = pad_sequences(X_test, maxlen=max_length, padding='post', truncating='post')

In [44]:
print('Encoded X Train\n', X_train, '\n')
print('Encoded X Test\n', X_test, '\n')
print('Maximum review length: ', max_length)

Encoded X Train
 [[   9    5   45 ...    1 1621  614]
 [ 130   68   10 ...  581   12 1991]
 [ 703 1264  111 ...    0    0    0]
 ...
 [ 760    8   25 ...    0    0    0]
 [   8  345   11 ...    0    0    0]
 [   8    5  510 ...   47    2  155]] 

Encoded X Test
 [[ 130   62  401 ...  791    3  632]
 [  10  104  674 ...  245 2497  389]
 [   1   17  881 ...    1  562    3]
 ...
 [ 418  140   65 ...    1  822    5]
 [  12  215   12 ...  595  494   32]
 [ 130  199  147 ...   88   16   43]] 

Maximum review length:  100


In [53]:
# Build LSTM Model for text classification

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Embedding, LSTM
from tensorflow.python.keras.layers.embeddings import Embedding

EMBEDDING_DIM = 100

print('Build model...')

model = Sequential()
model.add(Embedding(5000, EMBEDDING_DIM, input_length = max_length))
model.add(LSTM(units=32,  dropout=0.2))
model.add(Dense(1, activation='sigmoid'))

model.compile(optimizer = 'RMSprop', loss = 'binary_crossentropy', metrics = ['accuracy'])

print('Summary of the built model...')
print(model.summary())

Build model...
Summary of the built model...
Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 100, 100)          500000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 32)                17024     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 33        
Total params: 517,057
Trainable params: 517,057
Non-trainable params: 0
_________________________________________________________________
None


In [54]:
history = model.fit(X_train, y_train, batch_size=128, epochs=5, verbose=2, validation_split=0.2)

Epoch 1/5
250/250 - 45s - loss: 0.5036 - accuracy: 0.7499 - val_loss: 0.3824 - val_accuracy: 0.8340
Epoch 2/5
250/250 - 42s - loss: 0.3599 - accuracy: 0.8479 - val_loss: 0.3767 - val_accuracy: 0.8317
Epoch 3/5
250/250 - 42s - loss: 0.3240 - accuracy: 0.8657 - val_loss: 0.3864 - val_accuracy: 0.8380
Epoch 4/5
250/250 - 42s - loss: 0.3026 - accuracy: 0.8756 - val_loss: 0.4061 - val_accuracy: 0.8108
Epoch 5/5
250/250 - 42s - loss: 0.2864 - accuracy: 0.8830 - val_loss: 0.4034 - val_accuracy: 0.8292


In [55]:
print('Testing...')
y_test = np.array(y_test)
loss, accuracy = model.evaluate(X_test, y_test, batch_size=128)

print('Test loss:', loss)
print('Test accuracy:', accuracy)
print("Accuracy: {0:.2%}".format(accuracy))

Testing...
79/79 [==============================] - 2s 28ms/step - loss: 0.3920 - accuracy: 0.8329
Test loss: 0.39204397797584534
Test accuracy: 0.8328999876976013
Accuracy: 83.29%


In [57]:
# Let us test some  samples
test_sample_1 = "This movie is fantastic! I really like it because it is so good!"
test_sample_2 = "Good movie!"
test_sample_3 = "Maybe I like this movie."
test_sample_4 = "Not to my taste, will skip and watch another movie"
test_sample_5 = "if you like action, then this movie might be good for you."
test_sample_6 = "Bad movie!"
test_sample_7 = "Not a good movie!"
test_sample_8 = "This movie really sucks! Can I get my money back please?"
test_samples = [test_sample_1, test_sample_2, test_sample_3, test_sample_4, test_sample_5, test_sample_6, test_sample_7, test_sample_8]

for each in test_samples:
  filtered = [preprocess_text(each)]
  tokenize_words = tokenizer.texts_to_sequences(filtered)
  tokenize_words = pad_sequences(tokenize_words, maxlen=max_length, padding='post', truncating='post')
  result = model.predict(tokenize_words)
  if result >= 0.7:
      print('positive == ',each)
  else:
      print('negative == ',each)

positive ==  This movie is fantastic! I really like it because it is so good!
negative ==  Good movie!
negative ==  Maybe I like this movie.
negative ==  Not to my taste, will skip and watch another movie
negative ==  if you like action, then this movie might be good for you.
negative ==  Bad movie!
negative ==  Not a good movie!
negative ==  This movie really sucks! Can I get my money back please?


In [42]:
# Summary
# optimizer: adams
# Test loss: 0.41735655069351196
# Test accuracy: 83.14%

# positive ==  This movie is fantastic! I really like it because it is so good!
# negative ==  Good movie!
# negative ==  Maybe I like this movie.
# negative ==  Not to my taste, will skip and watch another movie
# negative ==  if you like action, then this movie might be good for you.
# negative ==  Bad movie!
# negative ==  Not a good movie!
# negative ==  This movie really sucks! Can I get my money back please?
